# SciBERT reimplementation 
We use allen-sciBERT weights for the huggingface library

In [1]:
!pip install transformers > transformers_installation.txt

In [1]:
from transformers import *
import torch

/home/vivoli/miniconda3/envs/arxiv-manipulation/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [3]:
MODEL_PATH = 'allenai/scibert_scivocab_uncased'
# MODEL_PATH = 'allenai/scibert_scivocab_cased'

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModel.from_pretrained(MODEL_PATH)

There are also other models/vocabs that can be used (specifically for SciBERT, from the huggingface library):

- scibert-scivocab-uncased (Recommended)
- scibert-scivocab-cased
- scibert-basevocab-uncased
- scibert-basevocab-cased

We, now, load some data (Langulage Modeling tasks such as STS and Keyphrase) just to see how they can be mixed together.

In [ ]:
tokenizer.vocab

In [17]:
DATA_PATH = '../data'

In [18]:
!ls $DATA_PATH

keyphrase  s2orc-full-20200705v1  s2orc-sample-20200705v1  sentence-tranformers
README.md  s2orc-mini		  scibert		   snli_1.0


In [19]:
!pip install scispacy > scispacy_installation.txt

In [20]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask
   
    

#Sentences we want sentence embeddings for
sentences = ['This framework generates embeddings for each input sentence',
             'Sentences are passed as a list of string.',
             'The quick brown fox jumps over the lazy dog.']

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModel.from_pretrained(MODEL_PATH)

#Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')

#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [21]:
sentence_embeddings

tensor([[ 0.3799,  0.6788, -0.1119,  ..., -0.2169, -0.9323, -0.0378],
        [-0.0321,  0.2644,  0.1901,  ...,  0.1506, -0.1151,  0.1622],
        [-0.8504, -0.1624, -0.6954,  ..., -0.1627, -0.2877, -0.5472]])